In [1]:
import pandas as pd
from datetime import datetime, timedelta

import json

import pickle

In [2]:
data = pd.read_csv("data/116",sep = "\t" , header = None)
data.head()

,0,1,2,3,4,5,6,7,8,9
0,GPS,粤E19204,116,-1,113.14631,23.02181,2015-07-01 00:01:30,-1,-1,2015-07-01 00:01:34
1,GPS,粤E19207,116,-1,113.09928,23.05066,2015-07-01 00:01:30,-1,-1,2015-07-01 00:01:34
2,GPS,粤E16952,116,-1,113.14620,23.02110,2015-07-01 00:01:25,-1,-1,2015-07-01 00:01:34
3,GPS,粤E19205,116,-1,113.14623,23.02135,2015-07-01 00:01:31,-1,-1,2015-07-01 00:01:35
4,GPS,粤E17022,116,-1,113.14618,23.02138,2015-07-01 00:01:30,-1,-1,2015-07-01 00:01:35


In [3]:
data = data[(data[4]> 112) & (data[4]<113.5)]
data = data[(data[5]> 22) & (data[5]<24)]

# remove invalid time points
seconds = [str(s).zfill(2)  for s in range(60)]
minutes = [str(s).zfill(2)  for s in range(60)]
hours = [str(s).zfill(2)  for s in range(24)]
months = [str(s).zfill(2)  for s in range(1,13)]

data = data[data[6].str[-2:].isin(seconds)]
data = data[data[6].str[-5:-3].isin(minutes)]
data = data[data[6].str[-8:-6].isin(hours)]
data = data[data[6].str[5:7].isin(months)]
data = data[data[6].str[:4].isin(["2015"])]
data = data.reset_index(drop=True)


In [4]:
# make a new index (to keep the raw_index)
df = data.reset_index()

In [5]:
df.columns = ["raw_index","GPS", "licence_plate_number","route","direction","longitude","latitude",
               "calibrated_time","velocity","angle","raw_time"]

In [3]:
# df.head()

In [15]:
add_speed_data = pd.DataFrame(columns=["raw_index","GPS", "licence_plate_number","route","direction","longitude","latitude",
               "calibrated_time","velocity","angle","raw_time","lon_cp","lat_cp","time_cp","delta_lon", "delta_lat","delta_time"])

# create a new column for the change in lat, lon and time since last data point
for plate in df["licence_plate_number"].unique():
    bus_df = df[df["licence_plate_number"] == plate]
    bus_df = bus_df.reset_index(drop = True)
    bus_df.loc[:,"lon_cp"] = bus_df.loc[:,"longitude"]
    bus_df.loc[:,"lat_cp"] = bus_df.loc[:,"latitude"]
    bus_df.loc[:,"time_cp"] = bus_df.loc[:,"calibrated_time"]
    bus_df.lon_cp = bus_df.lon_cp.shift(1)
    bus_df.lat_cp = bus_df.lat_cp.shift(1)
    bus_df.time_cp = bus_df.time_cp.shift(1)
    bus_df.loc[:,"delta_lon"] = bus_df.loc[:,"longitude"] -  bus_df.loc[:,"lon_cp"]
    bus_df.loc[:,"delta_lat"] = bus_df.loc[:,"latitude"] -  bus_df.loc[:,"lat_cp"]
    bus_df.loc[:,"delta_time"] = pd.to_datetime(bus_df.loc[:,"calibrated_time"]) -  pd.to_datetime(bus_df.loc[:,"time_cp"])
    add_speed_data = pd.concat([add_speed_data, bus_df])
    print str(plate)

add_speed_data.head()

粤E19204
粤E19207
粤E16952
粤E19205
粤E17022
粤E17035
粤E16977
粤E17002
粤E17027
粤E17005
粤E16979
粤E17031
粤E16963
粤E17033
粤E17180
粤E17021
粤E17017
粤E19212
粤E17012
粤E16960
粤E19209
粤E17032
粤E17039
粤E17169


,raw_index,GPS,licence_plate_number,route,direction,longitude,latitude,calibrated_time,velocity,angle,raw_time,lon_cp,lat_cp,time_cp,delta_lon,delta_lat,delta_time
0,0,GPS,粤E19204,116,-1,113.14631,23.02181,2015-07-01 00:01:30,-1,-1,2015-07-01 00:01:34,NaN,NaN,NaN,NaN,NaN,NaT
1,19,GPS,粤E19204,116,-1,113.14631,23.02181,2015-07-01 00:01:35,-1,-1,2015-07-01 00:01:39,113.14631,23.02181,2015-07-01 00:01:30,0.0,0.0,00:00:05
2,39,GPS,粤E19204,116,-1,113.14631,23.02181,2015-07-01 00:01:40,-1,-1,2015-07-01 00:01:44,113.14631,23.02181,2015-07-01 00:01:35,0.0,0.0,00:00:05
3,57,GPS,粤E19204,116,-1,113.14631,23.02181,2015-07-01 00:01:45,-1,-1,2015-07-01 00:01:47,113.14631,23.02181,2015-07-01 00:01:40,0.0,0.0,00:00:05
4,82,GPS,粤E19204,116,-1,113.14631,23.02181,2015-07-01 00:01:50,-1,-1,2015-07-01 00:01:54,113.14631,23.02181,2015-07-01 00:01:45,0.0,0.0,00:00:05


In [ ]:
complete_df = add_speed_data[["raw_index","route","licence_plate_number","calibrated_time","longitude","latitude", "delta_lon", "delta_lat","delta_time"]]

complete_df.to_csv("data/116_edit",sep = "\t")